In [81]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": (
                "Given the name of a company returns its ticker symbol"
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": (
                "Given a ticker symbol (i.e AAPL) returns the company's income"
                " statement."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": (
                "Given a ticker symbol (i.e AAPL) returns the company's"
                " balance sheet."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": (
                "Given a ticker symbol (i.e AAPL) returns the performance of"
                " the stock for the last 100 days."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [82]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
assistant_id = "asst_7Hy1vJizAZ25onqlFeuyfDTB"

In [91]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_TXbIpJ6A0HkaPDku5r9giXeZ', created_at=1735127410, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [93]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_J4mznilfmBLjL09I3ZNxDmiW', assistant_id='asst_7Hy1vJizAZ25onqlFeuyfDTB', cancelled_at=None, completed_at=None, created_at=1735127431, expires_at=1735128031, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_TXbIpJ6A0HkaPDku5r9giXeZ', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(

In [84]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(
            f"Calling function: {function.name} with arg {function.arguments}"
        )
        outputs.append(
            {
                "output": functions_map[function.name](
                    json.loads(function.arguments)
                ),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [98]:
get_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


[{'output': '"{\\"1706659200000\\":{\\"Tax Effect Of Unusual Items\\":-202400000.0,\\"Tax Rate For Calcs\\":0.16,\\"Normalized EBITDA\\":11223000000.0,\\"Total Unusual Items\\":-1265000000.0,\\"Total Unusual Items Excluding Goodwill\\":-1265000000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":4136000000.0,\\"Reconciled Depreciation\\":3959000000.0,\\"Reconciled Cost Of Revenue\\":8541000000.0,\\"EBITDA\\":9958000000.0,\\"EBIT\\":5999000000.0,\\"Normalized Income\\":5198600000.0,\\"Net Income From Continuing And Discontinued Operation\\":4136000000.0,\\"Total Expenses\\":28858000000.0,\\"Total Operating Income As Reported\\":5011000000.0,\\"Diluted Average Shares\\":984000000.0,\\"Basic Average Shares\\":974000000.0,\\"Diluted EPS\\":4.2,\\"Basic EPS\\":4.25,\\"Diluted NI Availto Com Stockholders\\":4136000000.0,\\"Net Income Common Stockholders\\":4136000000.0,\\"Net Income\\":4136000000.0,\\"Net Income Including Noncontrolling Interests\\":4136000000.0,\\"Net Inco

In [99]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_J4mznilfmBLjL09I3ZNxDmiW', assistant_id='asst_7Hy1vJizAZ25onqlFeuyfDTB', cancelled_at=None, completed_at=None, created_at=1735127431, expires_at=1735128031, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1735127447, status='queued', thread_id='thread_TXbIpJ6A0HkaPDku5r9giXeZ', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefin

In [101]:
get_run(run.id, thread.id).status

'completed'

In [102]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: To determine if Salesforce's stock (CRM) is a good investment, let's take a closer look at its financials and stock performance.

### Financial Overview
#### Income Statement
- **Total Revenue (most recent year):** $34.86 billion
- **Gross Profit:** $26.32 billion
- **Operating Income:** $5.00 billion
- **Net Income:** $4.14 billion
- **Diluted EPS:** $4.20

Salesforce has shown strong revenue with significant gross profit, indicating effective cost management. The operating income and net income also suggest positive profitability.

#### Balance Sheet
- **Total Assets:** $99.82 billion
- **Total Liabilities:** $40.18 million
- **Stockholders' Equity:** $59.65 billion
- **Total Debt:** $12.59 billion

The company's debt levels are manageable relative to its total assets and equity, indicating a solid financial position.

### Stock Performance (Last 100 Days)
- **Recent Closing Price:** Approximately $344.43
- **52-we